# AeroLunar - Data Cleaning

*Datasets from:*
 
*https://data.gov.hk/en-data/dataset/hk-epd-airteam-past-record-of-air-quality-health-index-en (air quality health index)*

*https://www.somacon.com/p570.php (moon phase)*

*https://cd.epic.epd.gov.hk/EPICDI/air/station/ (pollutants)*

*Note:*

- There is no available data in 2024, only Jan to Mar in 2023 
- and only Dec in 2013 because the Hong Kong Environmental Protection Department migrated the index system from Air Pollution Index (API) to a more health-focused Air Quality Health Index (AQHI) in 2013
- So we use the data from **2014 to 2022 (9 years)** since inconsistency in timeframe can easily cause error in time-series analysis

*Main context in [main.ipynb](main.ipynb)*

***Outline:***
1. **AQHI dataset**
	- 1.1 Check common columns (for the location of the stations)
	- 1.2 Merge the files into one dataframe
	- 1.3 Data type conversion
	- 1.4 Missing values handling

2. **Moon phase dataset**
	- 2.1 Merge the into one dataframe
	- 2.2 Data preprocessing

3. **Pollutants dataset**
	- 3.1 Merge the files
	- 3.2 Data type conversion
	- 3.3 Missing values handling
	- 3.4 Further inconsistencies handling (inconsistency in "Shatin" and "Sha Tin" & missing values in "Tap Mun")

4. **Overview and export the dataset**

In [1]:
import numpy as np
import pandas as pd
import os

## 1. AQHI dataset

### 1.1 Check common columns (for the location of the stations)

In [2]:
current_directory = os.getcwd()
common_columns = None
unique_columns = None

for year in range(2014, 2023):
    for month in range(1, 13):
        
        # file path -> curr_dir/raw_data/aqhi/20XX/20XX (X).csv
        file_name = os.path.join(current_directory,'raw_data/aqhi', f'{year}', f'{year} ({month}).csv')

        # skip the seven lines of metadata
        df = pd.read_csv(file_name, skiprows=7)
        
        # find the common columns and add them to a pandas Index object
        current_columns = pd.Index(df.columns)
                
        if common_columns is None:
            common_columns = current_columns
        else:
            common_columns = common_columns.intersection(current_columns) 
        
        # find all the unique columns 
        if unique_columns is None:
            unique_columns = current_columns
        else:
            unique_columns = unique_columns.union(current_columns) 

print(f'Common columns: \n {common_columns.values} \n')

# print the excluded columns as (unique - common)
excluded_columns = unique_columns.difference(common_columns)
print(f'Excluded columns: \n {excluded_columns.values}')
        

Common columns: 
 ['Date' 'Hour' 'Central/Western' 'Eastern' 'Kwun Tong' 'Sham Shui Po'
 'Kwai Chung' 'Tsuen Wan' 'Yuen Long' 'Tuen Mun' 'Tung Chung' 'Tai Po'
 'Sha Tin' 'Tap Mun' 'Causeway Bay' 'Central' 'Mong Kok'] 

Excluded columns: 
 ['North' 'Southern' 'Tseung Kwan O']


### 1.2 Merge the files 

In [3]:
csv_files = []

for year in range(2014, 2023):
    for month in range(1, 13):
        
        file_name = os.path.join(current_directory,'raw_data/aqhi', f'{year}', f'{year} ({month}).csv')

        # read only the common columns
        df = pd.read_csv(file_name, skiprows=7)[common_columns]
        csv_files.append(df)

# combine all the files into one df
all_data = pd.concat(csv_files, ignore_index=True)
all_data.head(5)


,Date,Hour,Central/Western,Eastern,Kwun Tong,Sham Shui Po,Kwai Chung,Tsuen Wan,Yuen Long,Tuen Mun,Tung Chung,Tai Po,Sha Tin,Tap Mun,Causeway Bay,Central,Mong Kok
0,2014-09-01,01,1,2,2,2,2,2,1,2,1,2,2,2,3,2,2
1,NaN,02,1,2,2,2,1,2,1,2,1,2,2,2,3,2,2
2,NaN,03,1,2,2,2,1,1,1,2,1,2,2,1,2,2,2
3,NaN,04,1,2,2,2,1,1,1,2,1,2,2,1,2,2,2
4,NaN,05,1,2,2,2,1,1,1,2,1,2,2,1,2,2,2


### 1.3 Data type conversion

In [4]:
air_df = all_data.copy()

# drop 'Daily Max' row
air_df = air_df[air_df['Hour'] != 'Daily Max']

# forward fill the date for easier manipulation -> group by day or month or year
air_df['Date'] = air_df['Date'].fillna(method = 'ffill')
air_df.head(5)

,Date,Hour,Central/Western,Eastern,Kwun Tong,Sham Shui Po,Kwai Chung,Tsuen Wan,Yuen Long,Tuen Mun,Tung Chung,Tai Po,Sha Tin,Tap Mun,Causeway Bay,Central,Mong Kok
0,2014-09-01,01,1,2,2,2,2,2,1,2,1,2,2,2,3,2,2
1,2014-09-01,02,1,2,2,2,1,2,1,2,1,2,2,2,3,2,2
2,2014-09-01,03,1,2,2,2,1,1,1,2,1,2,2,1,2,2,2
3,2014-09-01,04,1,2,2,2,1,1,1,2,1,2,2,1,2,2,2
4,2014-09-01,05,1,2,2,2,1,1,1,2,1,2,2,1,2,2,2


In [5]:
locations = common_columns.drop(['Date', 'Hour'])

# remove the asterisks
air_df[locations] = air_df[locations].replace({'\*': ''}, regex=True)

# convert '10+' to 11
air_df[locations] = air_df[locations].replace({'10\+': '11'}, regex=True)

# convert 'Date' to datetime
air_df['Date'] = pd.to_datetime(air_df['Date'])

# convert locations to int and replace the NaN to np.nan for doing interpolation to fill the missing values later
air_df[locations] = air_df[locations].apply(
  lambda col: pd.to_numeric(col.replace({'NaN': np.nan}), errors='coerce')) 

# convert 'Hour' to int
air_df['Hour']= air_df['Hour'].astype(int) 

### 1.4 Null values handling

In [6]:
# count missing value per row before interpolation
na_count = air_df.isna().sum(axis=1)  
na_rows = (na_count > 0).sum()   
print(f'Before interpolation, there are {na_rows} rows contain at least one missing value.') 

# doing interpolation to fill the missing values
air_df[locations] = air_df[locations].interpolate(method = 'linear')

# count missing value per row after interpolation
na_count = air_df.isna().sum(axis=1)  
na_rows = (na_count > 0).sum()   
print(f'After interpolation, there are {na_rows} rows contain at least one missing value.') 

Before interpolation, there are 4058 rows contain at least one missing value.
After interpolation, there are 0 rows contain at least one missing value.


In [7]:
# melt the dataframe and create a new column for the locations in order to take them as features
air_df = air_df.melt(id_vars=['Date', 'Hour'], var_name='Location', value_name='AQHI')
air_df.head(5)

,Date,Hour,Location,AQHI
0,2014-09-01,1,Central/Western,1.0
1,2014-09-01,2,Central/Western,1.0
2,2014-09-01,3,Central/Western,1.0
3,2014-09-01,4,Central/Western,1.0
4,2014-09-01,5,Central/Western,1.0


## 2. Moon phase dataset

### 2.1 Merge the files 

In [8]:
csv_files = []

for year in range(2014, 2023):  
  file_name = os.path.join(current_directory,'raw_data/moonphase', f'moon-phases-{year}-Asia_Hong_Kong.csv')

  # read only the common columns
  df = pd.read_csv(file_name)
  csv_files.append(df)

# combine all the files into one df
all_data = pd.concat(csv_files, ignore_index=True)
all_data.head(5)

,date,time,phase,phaseid,datetime,timestamp,friendlydate,timezone
0,01/01/2014,07:14 PM,New Moon,1,2014-01-01 19:14:00,1388574840,"January 1, 2014",Asia/Hong_Kong
1,01/08/2014,11:39 AM,First Quarter,2,2014-01-08 11:39:00,1389152340,"January 8, 2014",Asia/Hong_Kong
2,01/16/2014,12:52 PM,Full Moon,3,2014-01-16 12:52:00,1389847920,"January 16, 2014",Asia/Hong_Kong
3,01/24/2014,01:19 PM,Last Quarter,4,2014-01-24 13:19:00,1390540740,"January 24, 2014",Asia/Hong_Kong
4,01/31/2014,05:38 AM,New Moon,1,2014-01-31 05:38:00,1391117880,"January 31, 2014",Asia/Hong_Kong


### 2.2 Data preprocessing

In [9]:
# drop irrelevant columns
moon_df = all_data.copy()
moon_df = moon_df[["date", "phase", "phaseid"]]

# rename "date" to "Date" for consistency
moon_df.rename(columns={'date': 'Date'}, inplace=True)

# convert to datetime 
moon_df['Date'] = pd.to_datetime(moon_df['Date'])

# merge moonphase with air quality index based on date column
merged_df = pd.merge(air_df, moon_df, how='outer') 
merged_df.tail(5)

,Date,Hour,Location,AQHI,phase,phaseid
1183315,2022-11-30,20,Mong Kok,2.0,First Quarter,2.0
1183316,2022-11-30,21,Mong Kok,2.0,First Quarter,2.0
1183317,2022-11-30,22,Mong Kok,2.0,First Quarter,2.0
1183318,2022-11-30,23,Mong Kok,2.0,First Quarter,2.0
1183319,2022-11-30,24,Mong Kok,2.0,First Quarter,2.0


In [10]:
# forward and backward fill the missing moon phase and moon phase id
merged_df['phase'] = merged_df['phase'].fillna(method = 'ffill').fillna(method = 'bfill')
merged_df['phaseid'] = merged_df['phaseid'].fillna(method = 'ffill').fillna(method = 'bfill')

# sort the data by Date, Hour in ascending order since it is sequential data
merged_df.sort_values(by=['Date', 'Hour'], inplace=True)
merged_df.head(5)

,Date,Hour,Location,AQHI,phase,phaseid
32040,2014-01-01,1,Central/Western,6.0,New Moon,1.0
32064,2014-01-01,1,Eastern,7.0,New Moon,1.0
32088,2014-01-01,1,Kwun Tong,9.0,New Moon,1.0
32112,2014-01-01,1,Sham Shui Po,8.0,New Moon,1.0
32136,2014-01-01,1,Kwai Chung,6.0,New Moon,1.0


## 3. Pollutants dataset

### 3.1 Merge the files

In [11]:
file_name_a = os.path.join(current_directory, 'raw_data/pollutants', 'pollu_14to18.csv')
file_name_b = os.path.join(current_directory, 'raw_data/pollutants', 'pollu_19to22.csv')

df_a = pd.read_csv(file_name_a, skiprows=11) # skip the metadata
df_b = pd.read_csv(file_name_b, skiprows=11)
csv_files = [df_a, df_b]

# combine all the files into one df
all_data = pd.concat(csv_files, ignore_index=True)
all_data.head(5)

,DATE,HOUR,STATION,SO2,NOX,NO2,CO,RSP,O3,FSP
0,1/1/2014,1,CAUSEWAY BAY,43,828,285,185,149,3,116
1,1/1/2014,2,CAUSEWAY BAY,54,771,249,190,157,3,125
2,1/1/2014,3,CAUSEWAY BAY,47,646,223,176,175,3,141
3,1/1/2014,4,CAUSEWAY BAY,48,533,191,248,141,3,113
4,1/1/2014,5,CAUSEWAY BAY,38,502,187,162,118,3,93


### 3.2 Data type conversion

In [12]:
po_df = all_data.copy()

# rename the columns for consistency
po_df.rename(columns={'DATE': 'Date', 'HOUR': 'Hour', 'STATION': 'Location'}, inplace=True)

# change the location name to title case for consistenc6
po_df['Location'] = po_df['Location'].str.title()

# convert the pollutants to int
pollutants = po_df.columns[3:].tolist()
po_df[pollutants] = po_df[pollutants].apply(
  lambda col: pd.to_numeric(col.replace({'N.A.': np.nan}), errors='coerce'))

# convert Hour to int
po_df['Hour']= po_df['Hour'].astype(int)

# convert Date to datetime
po_df['Date'] = pd.to_datetime(po_df['Date'], format='%d/%m/%Y')

po_df.head(5)

,Date,Hour,Location,SO2,NOX,NO2,CO,RSP,O3,FSP
0,2014-01-01,1,Causeway Bay,43.0,828.0,285.0,185.0,149.0,3.0,116.0
1,2014-01-01,2,Causeway Bay,54.0,771.0,249.0,190.0,157.0,3.0,125.0
2,2014-01-01,3,Causeway Bay,47.0,646.0,223.0,176.0,175.0,3.0,141.0
3,2014-01-01,4,Causeway Bay,48.0,533.0,191.0,248.0,141.0,3.0,113.0
4,2014-01-01,5,Causeway Bay,38.0,502.0,187.0,162.0,118.0,3.0,93.0


### 3.3 Null values handling

In [13]:
# handling the missing values

# count missing value per row before interpolation
na_count = po_df.isna().sum(axis=1)  
na_rows = (na_count > 0).sum()   
print(f'Before interpolation, there are {na_rows} rows contain at least one missing value.') 

# doing interpolation to fill the missing values
po_df[pollutants] = po_df[pollutants].interpolate(method = 'linear')

# count missing value per row after interpolation
na_count = po_df.isna().sum(axis=1)  
na_rows = (na_count > 0).sum()   
print(f'After interpolation, there are {na_rows} rows contain at least one missing value.') 

Before interpolation, there are 599678 rows contain at least one missing value.
After interpolation, there are 0 rows contain at least one missing value.


### 3.4 Further inconsistencies handling (inconsistency in "Shatin" and "Sha Tin" & missing values in "Tap Mun")

In [14]:
# check the inconsistency in location names with merged_df
merged_locat = set(merged_df['Location'].unique())
po_locat = set(po_df['Location'].unique())

print(merged_locat - po_locat)
print(po_locat - merged_locat)

{'Sha Tin'}
{'Shatin'}


In [15]:
po_df['Location'] = po_df['Location'].replace({'Shatin': 'Sha Tin'})

merged_df = pd.merge(merged_df, po_df, how='outer') 
merged_df.head(5)

,Date,Hour,Location,AQHI,phase,phaseid,SO2,NOX,NO2,CO,RSP,O3,FSP
0,2014-01-01,1,Central/Western,6.0,New Moon,1.0,41.0,111.000000,104.0,90.001010,126.0,34.0,94.0
1,2014-01-01,1,Eastern,7.0,New Moon,1.0,50.0,89.003674,175.0,134.250757,107.0,8.0,71.0
2,2014-01-01,1,Kwun Tong,9.0,New Moon,1.0,52.0,792.000000,217.0,222.750252,150.0,5.5,111.0
3,2014-01-01,1,Sham Shui Po,8.0,New Moon,1.0,41.0,529.000000,178.0,96.999909,140.0,5.0,102.0
4,2014-01-01,1,Kwai Chung,6.0,New Moon,1.0,27.0,250.000000,136.0,178.500505,115.0,5.0,91.0


In [16]:
# check missing values after merging
null_values = merged_df['SO2'].isnull()

# display the rows with null values
temp = merged_df[null_values]
temp

,Date,Hour,Location,AQHI,phase,phaseid,SO2,NOX,NO2,CO,RSP,O3,FSP
251651,2015-12-01,1,Tap Mun,2.001342,First Quarter,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251666,2015-12-01,2,Tap Mun,2.002685,First Quarter,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251681,2015-12-01,3,Tap Mun,2.004027,First Quarter,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251696,2015-12-01,4,Tap Mun,2.005369,First Quarter,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251711,2015-12-01,5,Tap Mun,2.006711,First Quarter,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
273896,2016-01-31,20,Tap Mun,5.973154,Full Moon,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273911,2016-01-31,21,Tap Mun,5.978523,Full Moon,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273926,2016-01-31,22,Tap Mun,5.983893,Full Moon,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273941,2016-01-31,23,Tap Mun,5.989262,Full Moon,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# the pollutants for Tap Mun is missing from Dec 2015 to Jan 2016
# drop the data for Tap Mun
merged_df = merged_df[merged_df['Location'] != 'Tap Mun']

## 4. Overview and export the dataset

In [18]:
merged_df

,Date,Hour,Location,AQHI,phase,phaseid,SO2,NOX,NO2,CO,RSP,O3,FSP
0,2014-01-01,1,Central/Western,6.0,New Moon,1.0,41.0,111.000000,104.0,90.001010,126.0,34.0,94.0
1,2014-01-01,1,Eastern,7.0,New Moon,1.0,50.0,89.003674,175.0,134.250757,107.0,8.0,71.0
2,2014-01-01,1,Kwun Tong,9.0,New Moon,1.0,52.0,792.000000,217.0,222.750252,150.0,5.5,111.0
3,2014-01-01,1,Sham Shui Po,8.0,New Moon,1.0,41.0,529.000000,178.0,96.999909,140.0,5.0,102.0
4,2014-01-01,1,Kwai Chung,6.0,New Moon,1.0,27.0,250.000000,136.0,178.500505,115.0,5.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183314,2022-12-31,24,Tai Po,3.0,First Quarter,2.0,3.0,80.000000,61.0,78.001027,43.0,7.0,34.0
1183315,2022-12-31,24,Sha Tin,3.0,First Quarter,2.0,6.0,35.000000,33.0,96.999259,29.0,51.0,19.0
1183317,2022-12-31,24,Causeway Bay,4.0,First Quarter,2.0,4.0,224.000000,83.0,91.000000,42.0,14.0,29.0
1183318,2022-12-31,24,Central,4.0,First Quarter,2.0,4.0,165.000000,81.0,89.000000,35.0,13.0,25.0


In [19]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1104432 entries, 0 to 1183319
Data columns (total 13 columns):
 #   Column    Non-Null Count    Dtype         
---  ------    --------------    -----         
 0   Date      1104432 non-null  datetime64[ns]
 1   Hour      1104432 non-null  int32         
 2   Location  1104432 non-null  object        
 3   AQHI      1104432 non-null  float64       
 4   phase     1104432 non-null  object        
 5   phaseid   1104432 non-null  float64       
 6   SO2       1104432 non-null  float64       
 7   NOX       1104432 non-null  float64       
 8   NO2       1104432 non-null  float64       
 9   CO        1104432 non-null  float64       
 10  RSP       1104432 non-null  float64       
 11  O3        1104432 non-null  float64       
 12  FSP       1104432 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int32(1), object(2)
memory usage: 113.8+ MB


In [20]:
# export
merged_df.to_csv('cleaned_data.csv', index=False)